In [65]:
# Imports and connection with db
import mysql.connector
import json
import requests
import time
import csv
import config
api_key = config.api_key
config.host
cnx = mysql.connector.connect(
                              host = config.host,
                              user = config.user,
                              passwd = config.passwd,
                              database = "Movie Project1"
                              )
cursor = cnx.cursor()
print(cnx)

In [66]:
# Get titles from csv
with open('Romcom_Movies.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    movie_titles = []
    for movie in reader:
        movie_titles.append(movie['title'])

In [67]:
movie_titles

['What Men Want',
 'Second Act',
 'Crazy Rich Asians',
 'Overboard (2018)',
 'Home Again',
 "Bridget Jones's Baby",
 'How to Be Single',
 'Trainwreck',
 'Blended',
 'About Last Night (2014)',
 'That Awkward Moment',
 'Don Jon',
 'The Big Wedding',
 'Silver Linings Playbook',
 "What to Expect When You're Expecting",
 'The Five-Year Engagement',
 'Think Like a Man',
 "New Year's Eve",
 'Crazy, Stupid, Love.',
 'Friends with Benefits',
 'Larry Crowne',
 'Monte Carlo',
 'Midnight in Paris',
 'Something Borrowed',
 'Jumping the Broom',
 'Arthur (2011)',
 'Just Go With It',
 'No Strings Attached',
 'How Do You Know',
 'Life as We Know It',
 'Easy A',
 'Going the Distance',
 'Sex and the City 2',
 'Letters to Juliet',
 'The Back-Up Plan',
 "She's Out of My League",
 'The Good Guy',
 "Valentine's Day",
 'Leap Year',
 "It's Complicated",
 'Did You Hear About the Morgans?',
 'The Ugly Truth',
 '(500) Days of Summer',
 'The Proposal',
 "My Best Friend's Girl",
 'Everybody Wants to Be Italian',
 '

In [68]:
len(movie_titles)

162

In [69]:
# API call
def all_results(api_key, movies):
    results = []
    for i in range(len(movies)):
        title = movies[i]
        url_params = {'term': title.replace(' ', '+')}
        url = f'http://www.omdbapi.com/?apikey={api_key}&t={title}'
        response = requests.get(url, params = url_params)
        results.append(response.json())
        time.sleep(1) # wait a second
    return results

In [70]:
test = all_results(api_key, movie_titles)

In [71]:
test

[{'Title': 'What Men Want',
  'Year': '2019',
  'Rated': 'R',
  'Released': '08 Feb 2019',
  'Runtime': '117 min',
  'Genre': 'Comedy, Fantasy, Romance',
  'Director': 'Adam Shankman',
  'Writer': 'Tina Gordon (screenplay by), Peter Huyck (screenplay by), Alex Gregory (screenplay by), Jas Waters (story by), Tina Gordon (story by), Josh Goldsmith (based on "What Women Want" screenplay by), Cathy Yuspa (based on "What Women Want" screenplay by), Josh Goldsmith (based on "What Women Want" story by), Cathy Yuspa (based on "What Women Want" story by), Diane Drake (based on "What Women Want" story by)',
  'Actors': 'Taraji P. Henson, Kristen Ledlow, Josh Brener, Kellan Lutz',
  'Plot': "A woman is boxed out by the male sports agents in her profession, but gains an unexpected edge over them when she develops the ability to hear men's thoughts.",
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'N/A',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYxNjE2NjIwOF5BMl5BanBnXkFtZTgw

In [72]:
# Extract just IMDb ratings and inserting into the designated table
def all_ratings(movies):
    all_data = []
    for movie in movies:
        if 'Title' in movie.keys():
            try:
                data = {'Title': movie['Title'],
                       'imdbRating': float(movie['imdbRating'])}
            except:
                continue
            cursor = cnx.cursor()
            try:
                cursor.execute("""INSERT INTO Reviews (title, ratings)
                                           VALUES (%s, %s)""", (data['Title'], data['imdbRating']))
            except:
                pass
            cnx.commit()
            all_data.append(data)
    return all_data

In [73]:
final_test = all_ratings(test)

In [74]:
final_test

[{'Title': 'What Men Want', 'imdbRating': 4.9},
 {'Title': 'Second Act', 'imdbRating': 5.8},
 {'Title': 'Crazy Rich Asians', 'imdbRating': 7.0},
 {'Title': 'Home Again', 'imdbRating': 5.8},
 {'Title': "Bridget Jones's Baby", 'imdbRating': 6.5},
 {'Title': 'How to Be Single', 'imdbRating': 6.1},
 {'Title': 'Trainwreck', 'imdbRating': 6.2},
 {'Title': 'Blended', 'imdbRating': 6.5},
 {'Title': 'That Awkward Moment', 'imdbRating': 6.2},
 {'Title': 'Don Jon', 'imdbRating': 6.6},
 {'Title': 'The Big Wedding', 'imdbRating': 5.6},
 {'Title': 'Silver Linings Playbook', 'imdbRating': 7.7},
 {'Title': "What to Expect When You're Expecting", 'imdbRating': 5.7},
 {'Title': 'The Five-Year Engagement', 'imdbRating': 6.2},
 {'Title': 'Think Like a Man', 'imdbRating': 6.6},
 {'Title': "New Year's Eve", 'imdbRating': 5.7},
 {'Title': 'Crazy, Stupid, Love.', 'imdbRating': 7.4},
 {'Title': 'Friends with Benefits', 'imdbRating': 6.6},
 {'Title': 'Larry Crowne', 'imdbRating': 6.1},
 {'Title': 'Monte Carlo',

In [75]:
len(final_test)

155

In [76]:
# Save ratings for each genre in separate csv
data_list = final_test

keys = data_list[0].keys()
with open('Romcom_Ratings.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_list)